In [1]:
import pandas as pd
import numpy as np
import random

# -------------------------
# STEP 1: Load Wallet Data
# -------------------------
wallets = pd.read_csv("wallet_id.csv")  # CSV containing "wallet_id" column

# -------------------------
# STEP 2: Mock Transaction Data
# -------------------------
# In a real-world scenario, you'd fetch data from Compound V2/V3 APIs or Etherscan.
# For demonstration, we simulate features for each wallet:
def fetch_wallet_features(wallet_id):
    return {
        "tx_count": random.randint(5, 200),           # Number of transactions
        "borrow_repay_ratio": random.uniform(0, 1),   # 0 = perfect repay, 1 = only borrows
        "liquidation_count": random.randint(0, 5),    # Number of times liquidated
        "avg_health_factor": random.uniform(0.5, 2.5),# Higher = safer
        "wallet_age_days": random.randint(30, 1000)   # Wallet age in days
    }

wallet_features = [fetch_wallet_features(w) for w in wallets["wallet_id"]]
features_df = pd.DataFrame(wallet_features)
features_df["wallet_id"] = wallets["wallet_id"]

# -------------------------
# STEP 3: Feature Normalization
# -------------------------
def min_max_normalize(series):
    return (series - series.min()) / (series.max() - series.min() + 1e-9)

norm_features = pd.DataFrame()
norm_features["tx_count"] = min_max_normalize(features_df["tx_count"])
norm_features["borrow_repay_ratio"] = features_df["borrow_repay_ratio"]  # already 0-1
norm_features["liquidation_rate"] = min_max_normalize(features_df["liquidation_count"])
norm_features["health_factor"] = min_max_normalize(features_df["avg_health_factor"])
norm_features["wallet_age"] = min_max_normalize(features_df["wallet_age_days"])

# -------------------------
# STEP 4: Risk Scoring Model
# -------------------------
# Define weights for risk indicators
weights = {
    "borrow_repay_ratio": 0.3,    # Higher ratio = riskier
    "liquidation_rate": 0.3,      # More liquidations = riskier
    "health_factor": -0.2,        # Higher health factor = safer
    "wallet_age": -0.1,           # Older wallets = safer
    "tx_count": -0.1              # More transactions = safer
}

# Calculate risk index
risk_index = (
    weights["borrow_repay_ratio"] * norm_features["borrow_repay_ratio"] +
    weights["liquidation_rate"] * norm_features["liquidation_rate"] +
    weights["health_factor"] * (1 - norm_features["health_factor"]) +
    weights["wallet_age"] * (1 - norm_features["wallet_age"]) +
    weights["tx_count"] * (1 - norm_features["tx_count"])
)

# Convert to score (0 to 1000)
scores = (1 - risk_index) * 1000
scores = np.clip(scores, 0, 1000)  # Ensure within bounds

# -------------------------
# STEP 5: Export Results
# -------------------------
final_scores = pd.DataFrame({
    "wallet_id": features_df["wallet_id"],
    "score": scores.astype(int)
})
final_scores.to_csv("wallet_scores.csv", index=False)

print("wallet_scores.csv generated successfully!")


wallet_scores.csv generated successfully!
